## Altmap Experiments
### Compare altmap to map eq using networkx

In [30]:
from matplotlib.ticker import (AutoMinorLocator)

from altmap.altmap_helpers.general import *

# show plots in separate window
%pylab

# init rc params
init_plt_params()
fig_dir = '../../../../'

Using matplotlib backend: Qt5Agg
Populating the interactive namespace from numpy and matplotlib


In [7]:
# compute essential cost function values for a barbell network with
# given clique size
def ring_of_cliques_cost(clique_size = 3, number_of_cliques = 5, print_output=False):
    nc = clique_size
    c = number_of_cliques
    
    m = (nc*(nc-1) + 2) * c / 2.0 # number of edges in the network
    p0 = (nc - 1) /(2*m) # stat prob for 'normal' nodes
    pc = nc / (2*m) # stat prob for the connecting nodes

    # initial cost
    J_init = c*(nc-1)*p0*np.log2(1.0-p0) + c*pc*np.log2(1.0-pc)
    
    # maximum independent sets cost
    p1 = c*p0; p2 = c*pc
    J_ind = (nc-2)*p1*np.log2(1.0-p1) + 2*p2*np.log2(1.0-p2)
    
    # ground truth cost
    pi = (nc-2)*p0+2*pc; pnoti=1.0-pi; pinoti=1.0/m; pii = pi - pinoti
    J_true = -c*(pii*np.log2(pii/pi**2) + pinoti*np.log2(pinoti/(pi*pnoti)))
    
    if print_output:
        print (f"\nRing of {c} cliques with nc = {nc} nodes per clique:\n")
        print (f"Each node a module - cost = {J_init}")
        print (f"Ground truth cost = {J_true}")
        print (f"Independent sets cost = {J_ind}\n")
    
    return J_init, J_true, J_ind

num_cliques = 64
nc_max = 30
nc_list = list(range(3, nc_max + 1))
J_init_list = np.zeros((len(nc_list), 1))
J_true_list = np.zeros_like(J_init_list)
J_ind_list = np.zeros_like(J_init_list)
J_altmap_list = np.zeros_like(J_init_list)
J_altmap_sci_list = np.zeros_like(J_init_list)
for i,nc in enumerate(nc_list):
    J_init_list[i], J_true_list[i], J_ind_list[i] = ring_of_cliques_cost(nc, number_of_cliques=num_cliques)
    
    # compute actual objectives achieved by altmap
    # G = nx.ring_of_cliques(num_cliques, nc)
    # G = nx.convert_node_labels_to_integers(G, first_label=1)
    # communities_found, num_communities_found,_ ,_ = infomap(G, altmap=True)
    # J_altmap_list[i] = altmap_cost(G, communities_found)
    # 
    # communities_found, num_communities_found,_ ,_ = infomap(G, altmap=True, init='sc', update_inputfile=False)
    # J_altmap_sci_list[i] = altmap_cost(G, communities_found)

In [8]:
plt.close('all')
fig, ax = plt.subplots(figsize=(12,9))
# ax.suptitle(f'Ring of {num_cliques} cliques - objective over clique size')
ax.plot(nc_list, -J_true_list, 'x--', label='Ground truth')
ax.plot(nc_list, -J_init_list, '^--', label='Each node as a module')
ax.plot(nc_list, -J_ind_list, 'o--', label='Maximum independet sets')
ax.plot(nc_list, -J_altmap_list, '.--', ms=8, lw=2, label='Implementation')
ax.plot(nc_list, -J_altmap_sci_list, '.--', ms=8, lw=2, label='Implementation with SCI')
ax.xaxis.set_minor_locator(AutoMinorLocator())
ax.tick_params(which='both', width=2)
ax.grid(which='both')
ax.set_xlabel('Nodes per clique $n_c$', labelpad=20)
ax.set_ylabel('Synthesizing Infomap objective $\mathcal{J}(m)$', labelpad=20)
ax.legend()
plt.tight_layout()

In [25]:
nc_list = [4, 8, 16, 23, 32, 37]
c_list = list(range(3, 50))

num_detected_sci_array = np.zeros((len(nc_list), len(c_list)))
for i, nc in enumerate(nc_list):
    print (f'Iteration {i+1}/{len(nc_list)}')
    for j, c in enumerate(c_list):        
        G = nx.ring_of_cliques(c, nc)
        G = nx.convert_node_labels_to_integers(G, first_label=1)
        _, num_found ,_ ,_ = infomap(G, altmap=True, init='sc')
        num_detected_sci_array[i,j] = num_found

Iteration 1/6
Spectral clustering finished in 0.009839999999996962 seconds.
Spectral clustering finished in 0.005061999999981026 seconds.
Spectral clustering finished in 0.031244999999984202 seconds.
Spectral clustering finished in 0.035241999999982454 seconds.
Spectral clustering finished in 0.047358000000002676 seconds.
Spectral clustering finished in 0.05432700000000068 seconds.
Spectral clustering finished in 0.026798999999982698 seconds.
Spectral clustering finished in 0.03464800000000423 seconds.
Spectral clustering finished in 0.05658999999999992 seconds.
Spectral clustering finished in 0.10647800000000984 seconds.
Spectral clustering finished in 0.054870999999991454 seconds.
Spectral clustering finished in 0.046575999999987516 seconds.
Spectral clustering finished in 0.20653599999999983 seconds.
Spectral clustering finished in 0.18398600000000442 seconds.
Spectral clustering finished in 0.06427500000000919 seconds.
Spectral clustering finished in 0.07488599999999224 seconds.
Sp

In [31]:
fig, ax = plt.subplots(figsize=(12,9))
ax.plot(c_list, c_list, '--',  label=f'Correct detection')
for i,_ in enumerate(nc_list):
    ax.plot(c_list, num_detected_sci_array[i,:], 'x--',  label=f'{nc_list[i]} nodes per clique')

ax.xaxis.set_minor_locator(AutoMinorLocator())
# ax.tick_params(which='both', width=2)
ax.grid(which='both')
ax.set_xlabel(r'True number of cliques $c$')
ax.set_ylabel(r'Detected number of cliques $\hat{c}$')
ax.legend()
plt.tight_layout()

In [32]:
fig_path = fig_dir + 'roc_sci_num_detected.pdf'
plt.savefig(fig_path, dpi=600, format='pdf')

In [33]:
num_cliques = 32
clique_size = 8
N = num_cliques * clique_size # num nodes
G = nx.ring_of_cliques(num_cliques, clique_size)
G = nx.convert_node_labels_to_integers(G, first_label=1)

# run community detection
# communities_found, num_communities_found,_,_ = infomap(G, altmap=False)
# communities_found, num_communities_found,_,_ = infomap(G, altmap=True)
communities_found, num_communities_found,_,_ = infomap(G, altmap=True, init='sc')

# print results
print (communities_found)
print (f'We found {num_communities_found} communities.')

# cost = altmap_cost(G, communities_found)
# print (f'Achieved cost L = {cost}')


Spectral clustering finished in 0.129620999999986 seconds.
OrderedDict([(1, 9), (2, 9), (3, 9), (4, 9), (5, 9), (6, 9), (7, 9), (8, 9), (9, 10), (10, 10), (11, 10), (12, 10), (13, 10), (14, 10), (15, 10), (16, 10), (17, 11), (18, 11), (19, 11), (20, 11), (21, 11), (22, 11), (23, 11), (24, 11), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 12), (42, 12), (43, 12), (44, 12), (45, 12), (46, 12), (47, 12), (48, 12), (49, 13), (50, 13), (51, 13), (52, 13), (53, 13), (54, 13), (55, 13), (56, 13), (57, 14), (58, 14), (59, 14), (60, 14), (61, 14), (62, 14), (63, 14), (64, 14), (65, 2), (66, 2), (67, 2), (68, 2), (69, 2), (70, 2), (71, 2), (72, 2), (73, 2), (74, 2), (75, 2), (76, 2), (77, 2), (78, 2), (79, 2), (80, 2), (81, 3), (82, 3), (83, 3), (84, 3), (85, 3), (86, 3), (87, 3), (88, 3), (89, 3), (90, 3), (91, 3), (92, 3), (93, 3), (94, 3), (95, 3), (96, 3), (97, 15), (98, 15), (99, 15), (10

In [34]:
plt.close('all')
plt.figure()
drawNetwork(G, communities_found)
plt.tight_layout()

In [29]:
# generate network
num_cliques = 4
clique_size = 3
N = num_cliques * clique_size # num nodes
G = nx.ring_of_cliques(num_cliques, clique_size)
G = nx.convert_node_labels_to_integers(G, first_label=1)

# initial
nodes_ids = list(range(1, N+1))
communities_init = dict(zip(nodes_ids, nodes_ids))

# ground truth
labels = [1] * clique_size
for c in range(2, num_cliques+1):
    labels += [c]*clique_size
communities_true = dict(zip(nodes_ids, labels))

# independent sets
labels = list(range(1,clique_size+1)) * num_cliques
communities_ind = dict(zip(nodes_ids, labels))


plt.close('all')

plt.figure(figsize=(5,8))
drawNetwork(G, communities_true)
plt.tight_layout()

plt.figure(figsize=(5,8))
drawNetwork(G, communities_init)
plt.tight_layout()

plt.figure(figsize=(5,8))
drawNetwork(G, communities_ind)
plt.tight_layout()

# plt.close('all')
# fig, axs = plt.subplots(1,3)
# fig.suptitle(f'Sample ring of cliques networks')
# 
# drawNetwork(G, communities_true, ax=axs[0])
# axs[0].set_xlabel('Ground truth')
# drawNetwork(G, communities_init, ax=axs[1])
# axs[1].set_xlabel('Initial partition')
# drawNetwork(G, communities_ind, ax=axs[2])
# axs[2].set_xlabel('Maximum independent sets')